In [1]:
import pygame

# --- setup --- (luôn luôn có những dòng này để setup)
pygame.init()
WIDTH, HEIGHT = 960, 540
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong — Stage 1")
clock = pygame.time.Clock() # dùng cho việc điều khiểm frame rate
font = pygame.font.Font(None, 48) # cập nhật loại font và size mình muốn (font, cỡ)

# --- constants ---
PADDLE_W, PADDLE_H = 16, 100
BALL_SIZE = 10
PADDLE_SPEED = 600      # px/sec
BALL_SPEED = 500        # base speed
SCORE_TO_WIN = 1

# --- state ---
left = pygame.Rect(40, HEIGHT//2 - PADDLE_H//2, PADDLE_W, PADDLE_H)
right = pygame.Rect(WIDTH - 40 - PADDLE_W, HEIGHT//2 - PADDLE_H//2, PADDLE_W, PADDLE_H)
ball = pygame.Rect(WIDTH//2 - BALL_SIZE//2, HEIGHT//2 - BALL_SIZE//2, BALL_SIZE, BALL_SIZE)

ball_vx, ball_vy = BALL_SPEED, BALL_SPEED * 0.4
score_l, score_r = 0, 0
running = True

def reset_ball(direction=1): 
    global ball_vx, ball_vy
    ball.center = (WIDTH//2, HEIGHT//2)
    ball_vx = direction * BALL_SPEED
    ball_vy = BALL_SPEED * 0.4

# --- loop ---
while running:
    dt = clock.tick(60) / 1000.0
    # Always scale movement by dt (pixels_per_second × dt)
    # Use clock.tick(target_fps)/1000.0 to get seconds.

    # 1) events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False # điều kiện dừng

    # 2) input (real-time)
    keys = pygame.key.get_pressed()
    # Left paddle (W/S)
    dy = (keys[pygame.K_s] - keys[pygame.K_w]) * PADDLE_SPEED * dt
    left.y += dy
    # Right paddle (UP/DOWN)
    dy = (keys[pygame.K_DOWN] - keys[pygame.K_UP]) * PADDLE_SPEED * dt
    right.y += dy

    # clamp paddles
    left.y = max(0, min(HEIGHT - PADDLE_H, left.y))
    right.y = max(0, min(HEIGHT - PADDLE_H, right.y))

    # 3) update ball
    ball.x += int(ball_vx * dt)
    ball.y += int(ball_vy * dt)

    # wall bounce (top/bottom)
    if ball.top <= 0 and ball_vy < 0:
        ball.top = 0
        ball_vy *= -1
    if ball.bottom >= HEIGHT and ball_vy > 0:
        ball.bottom = HEIGHT
        ball_vy *= -1

    # paddle collisions (simple AABB)
    if ball.colliderect(left) and ball_vx < 0:
        ball.left = left.right
        ball_vx *= -1
        # add a little "spin" based on hit position
        offset = (ball.centery - left.centery) / (PADDLE_H/2)
        ball_vy = (BALL_SPEED * 0.8) * max(-1, min(1, offset))

    if ball.colliderect(right) and ball_vx > 0:
        ball.right = right.left
        ball_vx *= -1
        offset = (ball.centery - right.centery) / (PADDLE_H/2)
        ball_vy = (BALL_SPEED * 0.8) * max(-1, min(1, offset))

    # scoring (left/right walls)
    if ball.right < 0:
        score_r += 1
        reset_ball(direction=1)
    elif ball.left > WIDTH:
        score_l += 1
        reset_ball(direction=-1)

    # 4) draw  Khởi tạo không gian trò chơi
    screen.fill((20, 22, 30))
    # middle line
    for y in range(0, HEIGHT, 30):
        pygame.draw.rect(screen, (80, 80, 90), (WIDTH//2 - 2, y, 4, 20))
    # paddles & ball
    pygame.draw.rect(screen, (220, 220, 240), left)
    pygame.draw.rect(screen, (220, 220, 240), right)
    pygame.draw.ellipse(screen, (220, 220, 240), ball)

    # score
    score_surf = font.render(f"{score_l}   {score_r}", True, (235, 235, 255))
    rect = score_surf.get_rect(center=(WIDTH//2, 40))
    screen.blit(score_surf, rect)

    pygame.display.flip()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
